In [1]:
import os

import numpy as np
import pandas as pd

from scipy.stats import fisher_exact
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
project_directory = '/Users/tstoeger/Dropbox/biomedical_data_science_day/'  # <-- modify

In [3]:
def complete_path(p):
    """
    Completes the path toward a file.
    
    Input:
        p     str, path relative to project path
    
    Output:
        p     str, complete path
    """
    p = os.path.join(project_directory, p)
    return p

In [4]:
def read_go_and_turn_to_annotation(taxon_id, subset):
    """
    Reads Gene Ontology (GO) annotation and formats them
    toward format anticipated by our enrichment tool.
    
    Input:
        taxon_id    int, NCBI taxonomy identifier 
                         (e.g.: 9606 for human)
        subset      int, 'Component', 'Process', 
                        or 'Function', or 'all'
    
    Output:
        gene_annotation  df, with columns
                         'gene_entrez' and 'annotation' where
                         'gene_entrez': int with Entre Gene Identifier
                         'annotation': str with name of annotation
    """
    
    # load tab-delimited file
    p = complete_path('ncbi_gene/gene2go.gz')   

    df = pd.read_csv(
        filepath_or_buffer=p, 
        sep='\t', 
        usecols=[
            '#tax_id', 
            'GeneID', 
            'Evidence', 
            'Qualifier', 
            'GO_term', 
            'Category'
        ])

    # filter according to taxon
    df = df.loc[df.loc[:, '#tax_id']==taxon_id, :]

    # filter according to sub-category of GO
    if subset in ['Component', 'Process', 'Component']:
        df = df.loc[df.loc[:, 'Category']==subset, :]
    elif subset == 'all':
        pass
    else:
        raise ValueError(
        'Chosen subset parameter is not supported.'
        )

    # Exclude annotations for which there is some 
    # negative evidence (e.g.: annotators found that
    # a plausible annotation actually does not occur)
    # Note that for a given pair of gene and annotation
    # there may be positive and negative evidence, but
    # all pairs should be removed if there is any negative
    # http://geneontology.org/docs/guide-go-evidence-codes/
    forbidden_evidence = [
        'ND'      # Negative evidence
    ]
    forbidden_qualifiers = [
        'NOT',
        'NOT colocalizes_with',
        'NOT contributes_to'
    ]

    forbidden_terms = df.loc[
        (df.loc[:, 'Evidence'].isin(forbidden_evidence)) | 
        (df.loc[:, 'Qualifier'].isin(forbidden_qualifiers)),
        ['GeneID', 'GO_term']
    ]
    forbidden_terms.loc[:, 'is_forbidden'] = True

    df = pd.merge(df, forbidden_terms, how='outer', indicator=True)
    df = df.loc[df['_merge']=='left_only', :]
    
    # Get future pairs of  'gene_entrez' and 'annotation',
    # while removing duplicates; and format them according
    # to the docstring (help/description ) of this function
    # 'GeneID' will become 'gene_entrez', and 'GO_term'
    # will become 'annotation'
    
    df = df.loc[:, ['GeneID', 'GO_term']].drop_duplicates()

    df = df.rename(columns={
        'GeneID': 'gene_entrez', 
        'GO_term': 'annotation'}).reset_index(drop=True)
    df['gene_entrez'] = df['gene_entrez'].astype(int)

    return df

In [5]:
taxon_id = 9606   # the NCBI taxonomy identifier for Homo sapiens
gene_annotation = read_go_and_turn_to_annotation(taxon_id, 'all')

In [6]:
gene_annotation.head()

,gene_entrez,annotation
0,1,platelet degranulation
1,1,extracellular region
2,1,extracellular space
3,1,platelet alpha granule lumen
4,1,secretory granule lumen


In [7]:
def functional_enrichment(
    hits, 
    background, 
    gene_annotation,
    only_genes_with_annotation=True
):
    """
    Tool to compute functional enrichments using Fisher's
    Exact test.
    
    Input:
        hits         list-like, gene_entrez of hits
        background   list-like, gene_entrez of background (all genes in experiment)
        gene_annotation   df, with columns
                         'gene_entrez' and 'annotation' where
                         'gene_entrez': int with Entre Gene Identifier
                         'annotation': str with name of annotation 
        only_genes_with_annotation  optional, default: True
                                    if True, hit and backround
                                    will be filtered for genes
                                    in gene_annotation
    
    Output:
        results  df, with index 'annotation' and columns
                    'fold_enrichment' log2 ratio of enrichment of annotation over backround
                    'pvalue' Fisher's exact test on enrichment of annotation
                    'bonferroni' Bonferroni corrected pvalue (multiple testing)
                    'benjamini_hochberg' Benjamini Hochberg corrected pvalue (multiple testing)
    """


    # Create three sets:
    # - hits: the hits from the study
    # - background: the user supplied list of considered genes
    # - not hits: genes in background but not hits
    hits = set(hits)
    background = set(background)
    not_hits = background - hits

    # Optionally, use filter to only consider genes
    # that have at least one annotation, e.g.: see
    # https://www.nature.com/articles/s41598-018-19333-x
    # on gene annotation bias
    if only_genes_with_annotation:
        unique_annotations = set(gene_annotation['gene_entrez'])

        hits = hits.intersection(unique_annotations)
        background = background.intersection(unique_annotations)
        not_hits = not_hits.intersection(unique_annotations)

    # Force an error if not all hits are in the background
    if any([hit not in background for hit in hits]):
        raise AssertionError(
            'At least one hit is not in the supplied background.'
        )

    # Restrict annotations to those that occur in the background
    gene_annotation = gene_annotation[gene_annotation['gene_entrez'].isin(background)]

    # Restrict annotations to those occuring a certain number
    # Rationale is to get rid of the bulk of annotations that only affect
    # a very small number of gens <-- note this is somewhat arbritrary
    # and subjective cutoff
    minimally_required_candidates_in_annotation = 2
    c = gene_annotation['annotation'].value_counts()
    gene_annotation = gene_annotation[gene_annotation['annotation'].isin(
        c[c >= minimally_required_candidates_in_annotation].index)]

    # Prepare to loop through annotations
    # First, set 'annotation' to become the index of gene_annotation
    gene_annotation = gene_annotation.set_index('annotation')
    # Second, get all unique 'annotation'
    annotations = gene_annotation.index.unique()
    # Third, initialize a dataframe with all unique annotations in
    # rows, and 'fold_enrichment', 'pvalue', 'bonferroni',
    # 'benjamini_hochberg' as columns
    results = pd.DataFrame(
        index=annotations,
        columns=[
            'fold_enrichment',
            'pvalue',
            'bonferroni',
            'benjamini_hochberg'],
    )

    # Loop through annotations
    for annotation in annotations:

        # Set up a contingency table containing 
        #   [hit_genes_in_annotation, not_hit_genes_in_annotation], 
        #   [hit_genes_not_in_annotation, not_hit_genes_not_in_annotation]
        # where each of above is total number of such genes
        # https://www.pathwaycommons.org/guide/primers/statistics/fishers_exact_test/

        genes_with_annotation = set(gene_annotation.loc[annotation, 'gene_entrez'])
        genes_without_annotation = background - genes_with_annotation

        hit_genes_in_annotation = len(hits.intersection(genes_with_annotation))
        not_hit_genes_in_annotation = len(not_hits.intersection(genes_with_annotation))

        hit_genes_not_in_annotation = len(hits.intersection(genes_without_annotation))
        not_hit_genes_not_in_annotation = len(not_hits.intersection(genes_without_annotation))

        # Perform Fisher's exact test
        _, pvalue = fisher_exact(
            [
                 [hit_genes_in_annotation, not_hit_genes_in_annotation], 
                 [hit_genes_not_in_annotation, not_hit_genes_not_in_annotation]],
            alternative='greater'
        )

        # Count number of annotated genes in hits and backround and compute
        # the enrichment among hits
        in_hits = len(hits.intersection(genes_with_annotation))
        in_background = len(background.intersection(genes_with_annotation))

        fract_of_hits_w_annot = in_hits / len(hits)
        fract_of_background_w_annot = in_background / len(background)

        if fract_of_hits_w_annot>0:
            enrichment = np.log2(
                fract_of_hits_w_annot / fract_of_background_w_annot)
        else:
            enrichment = -np.inf

        results.loc[annotation, 'fold_enrichment'] = enrichment
        results.loc[annotation, 'pvalue'] = pvalue

    # Add corrections for multiple testing
    results.loc[:, 'bonferroni'] = multipletests(
        results.loc[:, 'pvalue'].values, method='bonferroni')[1]
    results.loc[:, 'benjamini_hochberg'] = multipletests(
        results.loc[:, 'pvalue'].values, method='fdr_bh')[1]

    # Sort results by pvalue
    results = results.sort_values('pvalue')

    return results

In [8]:
# Get list of hits and background genes to be considered (all in experiment)
# in this case we want to know, which annotations are carried by the 
# genes that most commonly appear in differential gene expression analysis
# Data is from Crow et al. 2019 "Predictablity of human gene expression",
# PNAS, https://www.pnas.org/content/116/13/6491


p = complete_path('candidate_genes/crow_et_al_2019/DE_Prior.txt')
study_with_genes = pd.read_csv(p, sep='\t')  # a table that sorts genes according to occurence
background_from_study = study_with_genes['Gene_EntrezID']
hits_from_study = study_with_genes.loc[:300, 'Gene_EntrezID']  # lets' treat the 300 most frequent genes as hits

In [9]:
go_enrichments = functional_enrichment(
    hits=hits_from_study, 
    background=background_from_study, 
    gene_annotation=gene_annotation,
    only_genes_with_annotation=False
)

In [10]:
go_enrichments.head()

,fold_enrichment,pvalue,bonferroni,benjamini_hochberg
annotation,,,,
extracellular space,2.32375,2.33358e-44,2.95548e-40,2.95548e-40
extracellular region,2.03754,1.60005e-37,2.02646e-33,1.01323e-33
collagen-containing extracellular matrix,2.89296,2.36128e-23,2.99057e-19,9.96855e-20
inflammatory response,2.72008,2.40895e-19,3.05093e-15,7.62734e-16
chemokine activity,4.67117,4.09498e-19,5.18629e-15,1.03726e-15


In [11]:
def read_gwas_and_turn_to_annotation():
    """
    Reads EBI GWAS annotation and formats them
    toward format anticipated by our enrichment tool.
    
    Output:
        gene_annotation  df, with columns
                         'gene_entrez' and 'annotation' where
                         'gene_entrez': int with Entre Gene Identifier
                         'annotation': str with name of trait
    """
    
    p = complete_path('ebi_gwas/full.tsv')
    df_gwas = pd.read_csv(
        p, 
        sep='\t', 
        usecols=['DISEASE/TRAIT', 'MAPPED_GENE']
    ).drop_duplicates()

    p = complete_path('ncbi_gene/Homo_sapiens.gene_info.gz')
    df_gene_info = pd.read_csv(p, sep='\t', usecols=['GeneID', 'Symbol'])

    # combine overly simplistically through merge: unambiguous single SNPs 
    # -> only for illustrative purposes of tutorial
    df_annotation = pd.merge(
        df_gene_info.rename(columns={'Symbol': 'MAPPED_GENE'}),
        df_gwas,
        how='inner'   
    ).rename(columns={
        'GeneID': 'gene_entrez',
        'DISEASE/TRAIT': 'annotation'})

    df_annotation = df_annotation[['gene_entrez', 'annotation']].drop_duplicates()

    return df_annotation

In [12]:
gwas_enrichments = functional_enrichment(
    hits=hits_from_study, 
    background=background_from_study, 
    gene_annotation=gene_annotation,
    only_genes_with_annotation=False
)

In [13]:

gwas_enrichments.head()

,fold_enrichment,pvalue,bonferroni,benjamini_hochberg
annotation,,,,
extracellular space,2.32375,2.33358e-44,2.95548e-40,2.95548e-40
extracellular region,2.03754,1.60005e-37,2.02646e-33,1.01323e-33
collagen-containing extracellular matrix,2.89296,2.36128e-23,2.99057e-19,9.96855e-20
inflammatory response,2.72008,2.40895e-19,3.05093e-15,7.62734e-16
chemokine activity,4.67117,4.09498e-19,5.18629e-15,1.03726e-15
